In [1]:
# check current kernel
import sys
sys.executable

'D:\\anaconda3\\python.exe'

In [2]:
import math

In [3]:
# class Point
class Point:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
    
    def distance_from_origin(self):
        return math.hypot(self.x, self.y)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return "Point({0.x!r}, {0.y!r})".format(self)
    
    def __str__(self):
        return "({0.x!r}, {0.y!r})".format(self)

In [4]:
a = Point()
repr(a)

'Point(0, 0)'

In [5]:
b = Point(3, 4)
str(b)

'(3, 4)'

In [6]:
b.distance_from_origin()

5.0

In [7]:
b.x = -19
str(b)

'(-19, 4)'

In [8]:
a == b, a != b

(False, True)

### special method __eq__()
By re-writing __eq__(), == operator is valid between two instances of a class.

Before the __eq__() method is implemented, we cannot compare any two instances of a class.
Thus, the result of a == b is always False. Besides, any instance is hashable(means it is unique and can be the key of dictionary, or the element of sets).

After the __eq__() menthod is implemented, instances of the class are no longer hashable.

### special method __repr__()

In [9]:
# class Circle
class Circle(Point):
    def __init__(self, radius, x = 0, y = 0):
        super().__init__(x, y)
        self.radius = radius
    
    def edge_distance_from_origin(self):
        return abs(self.distance_from_origin() - self.radius)

    def area(self):
        return math.pi * (self.radius ** 2)
    
    def circumference(self):
        return 2 * math.pi * self.radius
    
    def __eq__(self, other):
        return super().__eq__(other) and self.radius == other.radius

    def __repr__(self):
        return "Circle({0.radius!r}, {0.x!r}, {0.y!r})".format(self)

    def __str__(self):
        return repr(self)

In [10]:
p = Point(28, 45)
c = Circle(5, 28, 45)
p.distance_from_origin(), c.distance_from_origin()

(53.0, 53.0)

# Restriction on access and modification of attributes of an instance from outside

## Use method: write get method and set method

In [11]:
# without restriction
class Student():
    def __init__(self, name, score):
        self.name = name
        self.score = score

bart = Student("Bart Simpson", 59)

In [12]:
# access to internal attributes
bart.name, bart.score

('Bart Simpson', 59)

In [13]:
## modification to internal attributes
bart.name = "Bart Aaron"
bart.score = 99
bart.name, bart.score

('Bart Aaron', 99)

In [14]:
# use private attributes (starting with __, not ending with __) to avoid access from outside directly
class Student():
    def __init__(self, name, score):
        self.__name = name
        self.__score = score
    
    def print_score(self):
        print("%s: %s" % (self.__name, self.__score)) ## access from inside is valid

In [15]:
bart = Student("Bart Simpson", 59)
# try to access the attribute __name: banned
bart.__name

AttributeError: 'Student' object has no attribute '__name'

In [16]:
# what if we need to access the internal attributes safely(read-only)?
# use method instead
class Student():
    def __init__(self, name, score):
        self.__name = name
        self.__score = score
    
    def print_score(self):
        print("%s: %s" % (self.__name, self.__score)) ## access from inside is valid

    def get_name(self):
        return self.__name
    
    def get_score(self):
        return self.__score
bart = Student("Bart Simpson", 59)

In [17]:
bart.get_name(), bart.get_score()

('Bart Simpson', 59)

In [18]:
# while direct access is still banned
bart.__name

AttributeError: 'Student' object has no attribute '__name'

In [19]:
# what if we need to modify the internal attributes safely(read-only)?
# use method instead. The setting method can have conditions.
class Student():
    def __init__(self, name, score):
        self.__name = name
        self.__score = score
    
    def print_score(self):
        print("%s: %s" % (self.__name, self.__score)) ## access from inside is valid

    def get_name(self):
        return self.__name
    
    def get_score(self):
        return self.__score
    
    def set_name(self, new_name):
        if isinstance(new_name, str) and len(new_name) > 0:
            self.__name = new_name
        else:
            raise ValueError("Bad name")
    
    def set_score(self, new_score):
        if new_score >=0 and new_score <= 100:
            self.__score = new_score
        else:
            raise ValueError("Bad score")
bart = Student("Bart Simpson", 59)

In [20]:
bart.get_name(), bart.get_score()

('Bart Simpson', 59)

In [21]:
bart.set_name("Bart Aaron")
bart.set_score(99)
bart.get_name(), bart.get_score()

('Bart Aaron', 99)

## Use decorator: Built-in function Property
As we can see above, if we use methods to get or set attributes, it's always looking like to deploy methods.

For e.g,

     Student_bart.get_name()

     Student_bart.set_score(99)

What if we want to make the method deploy more close to getting and setting attributes?

For e.g,

     Student_bart.name,
     
     Student_bart.score = 99

In [22]:
# New Circle
# 我们重新设计Circle类，这次我们用私有的__radius作为attribute(这样__radius受保护，无法从外界access或modify)
# 把radius设置成特性(characteristic or property, 本质是仅靠自身self就可以确定的量，无需另外参数，面积、周长都是)
# 设置特性：意思就是把一个内禀的方法(a method only takes self as input), 用@property修饰器修饰。那么该方法就成为了一个特性。
# 所以特性本质就是一个方法，它可以从外部调用（看作attribute调用而不是当方法调用）。
class Circle(Point):
    def __init__(self, radius, x = 0, y = 0):
        super().__init__(x, y)
        self.__radius = radius
    
    @property #@property修饰一个属性(一般是私有属性)的getter方法时，等价于 radius = property(radius)
    def radius(self): #私有属性的getter方法
        return self.__radius
    
    # 用@property修饰一个私有属性的getter方法之后，可以用其衍生的修饰器修饰该私有属性的setter方法和deleter方法(与getter方法同名)。
    @radius.setter #衍生自radius，这个__radius的getter方法。注意修饰器名字为getter方法的名字.setter。
    def radius(self, new_radius): #私有属性的setter方法，注意函数名与该私有属性的getter方法相同。修饰器会自动处理重名。
        assert new_radius > 0, "radius must be positive"
        self.__radius = new_radius
    
    # 可以看出，edge_distance_from_origin也是一个内禀的方法(只有self输入)，所以它也可以被property修饰成一个特性。
    # 但显然我们不能set这个edge_distance_from_origin，所以它应该是只读的。这种情况下，只修饰getter方法，它就是只读的了。
    @property
    def edge_distance_from_origin(self):
        return abs(self.distance_from_origin() - self.radius)

    @property
    def area(self):
        return math.pi * (self.radius ** 2)
    
    @property
    def circumference(self):
        return 2 * math.pi * self.radius
    
    def __eq__(self, other):
        return super().__eq__(other) and self.radius == other.radius

    def __repr__(self):
        return "Circle({0.radius!r}, {0.x!r}, {0.y!r})".format(self)

    def __str__(self):
        return repr(self)

In [23]:
c = Circle(5, 3, 4)
c.radius #调用私有属性的getter方法，因为被修饰过了，所以不需要括号当method调用，像attribute读取就行

5

In [24]:
c.radius = 6 #调用私有属性的setter方法，因为被修饰过了，所以看起来像赋值而不是method调用
c

Circle(6, 3, 4)

In [25]:
c.edge_distance_from_origin #调用一个内禀方法，因为被修饰过了，所以不需要括号当method调用，像attribute读取就行

1.0

## Property总结  
设置特性：意思就是把一个内禀的方法(a method only takes self as input), 用@property修饰器修饰。那么该方法就成为了一个特性。  
1. @property修饰一个属性(一般是私有属性)的getter方法  
2. 用@property修饰一个私有属性的getter方法之后，可以用其衍生的修饰器修饰该私有属性的setter方法和deleter方法(与getter方法同名)。修饰器会自动处理重名。  
3. 可以不设置该私有属性的setter方法。这样就不能用赋值的方法set该私有属性，要调用专门的method来set。这样该属性看上去就是只读的了。

## Decorator修饰器
不带参数使用修饰器  
@decorater  
def some_func(args):suite  
当使用some_func时：  
output = some_func(args)  
实际上是：  
some_func = decorater(some_func) #decorater(some_func)输出了一个修饰后的函数，其被some_func这个函数名重新引用了。  
output = some_func(args)

In [26]:
# 希望限制discriminant函数的输出为非负，不然assertion
def positive_result(function):#修饰器, 以一个函数作为输入
    def wrapper(*args, **kwargs):
        result = function(*args, **kwargs) #这样可以运行function, 不论如何安排输入
        assert result >= 0, function.__name__ + "() result isn't >= 0" #修饰的部分
        return result
    # 如何直接return wrapper 的话，被修饰后的discriminant函数名实际上引用了上面的wrapper函数。
    # 于是被修饰后的discriminant函数的docstring, 即__name__和__doc__属性是wrapper的属性
    # 如果不需要用到这些属性，那也无所谓。但有时候使用discriminant时，需要docstring。
    wrapper.__name__ = function.__name__
    wrapper.__doc__ = function.__doc__
    return wrapper # 修改wrapper的docstring之后再return，这样修饰后的discriminant函数就和原discriminant几乎一样了

@positive_result
def discriminant(a, b, c):
    return b**2-4*a*c

带参数使用修饰器  
@decorater(deco_args)  
def some_func(args):suite  
当使用some_func时：  
output = some_func(args)  
实际上是：  
some_func = decorater(deco_args)(some_func) #decorator(deco_args)先输出一个修饰器函数，该修饰器函数修饰some_func之后输出的新函数对象，被some_func这个函数名重新引用了  
output = some_func(args)

In [27]:
# 希望限制percent函数的输出在minimun和maximum之间，超出上下限就返回上下限。
# bounded是一个带参数的修饰器
def bounded(minimum, maximun): #它应该return一个修饰器函数
    def decorator(function): ##修饰器函数，在它内部要重新写一个wrapper来执行function然后返回修饰后的wrapper
        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            if result < minimum:
                result = minimum
            if result > maximun:
                result = maximun
            return result
        # 修改wrapper的docstring使之与输入function保持一致
        wrapper.__name__ = function.__name__
        wrapper.__doc__ = function.__doc__
        return wrapper
    return decorator #返回修饰器函数

@bounded(0, 100) #每次修饰的时候就要输入修饰器的参数
def percent(amount, total):
    return amount/total*100

percent(-1, 2), percent(10, 20), percent(20, 10), percent.__name__

(0, 50.0, 100, 'percent')

        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            if result < minimum:
                result = minimum
            if result > maximun:
                result = maximun
            return result
        # 修改wrapper的docstring使之与输入function保持一致
        wrapper.__name__ = function.__name__
        wrapper.__doc__ = function.__doc__
        return wrapper
以上是修饰输入的function的过程，定义一个wrapper函数，先在其内部运行function，修饰function的结果之后输出修饰结果，然后修改wrapper的docstring使之与function保持一致，最后return wrapper。  
我们发现，“修改一个函数的docstring，然后return该函数”本身就是一个修饰过程。  
也就是说，我们可以使用修饰器来修饰wrapper函数，来使得输出的wrapper函数的docstring与function的docstring一致。所以这是一个带参数的修饰器。  
模块functools的functools.wraps修饰器可以实现这一功能。 
 
以下重写bounded修饰器：

In [28]:
import functools
# 希望限制percent函数的输出在minimun和maximum之间，超出上下限就返回上下限。
# bounded是一个带参数的修饰器
def bounded(minimum, maximun): #它应该return一个修饰器函数
    def decorator(function): ##修饰器函数，在它内部要重新写一个wrapper来执行function然后返回修饰后的wrapper
        @functools.wraps(function) #修改wrapper的docstring使之与输入function保持一致
        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            if result < minimum:
                result = minimum
            if result > maximun:
                result = maximun
            return result
        # 修改wrapper的docstring使之与输入function保持一致
        # wrapper.__name__ = function.__name__
        # wrapper.__doc__ = function.__doc__
        return wrapper
    return decorator #返回修饰器函数

@bounded(0, 100) #每次修饰的时候就要输入修饰器的参数
def percent(amount, total):
    return amount/total*100

percent(-1, 2), percent(10, 20), percent(20, 10), percent.__name__

(0, 50.0, 100, 'percent')

In [30]:
# 小练习，编写一个修饰器log，使得经它修饰后的任一个函数，在运行时都先打印 text + 函数名。text是一个变量。
def log(text):
    def decorator(function):
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            print(text + function.__name__) #修饰function
            output = function(*args, **kwargs)
            return output
        return wrapper ## wrapper是修饰后的function
    return decorator

@log("call: ")
def sum_diff(a, b):
    return (a+b, a-b)

@log("execute: ")
def now(month, day):
    print("Today is %s, %s" % (str(month), str(day)))

In [31]:
sum_diff(2, 3)

call: sum_diff


(5, -1)